### compare csv file before ingest


In [2]:
# run in the same environment you used before
import pandas as pd
df_old = pd.read_csv("23112025affiliate-98__TicketSqueeze-Events.csv", dtype=str).fillna("")
df_new = pd.read_csv("30112025affiliate-98__TicketSqueeze-Events.csv", dtype=str).fillna("")
key = "event_id"   # adjust if different

len_old = len(df_old)
len_new = len(df_new)

old_keys = set(df_old[key])
new_keys = set(df_new[key])

added = new_keys - old_keys
removed = old_keys - new_keys
common = old_keys & new_keys

print(len_old, len_new)
print("Added", len(added))
print("Removed", len(removed))
print("Common", len(common))

# check uniqueness
print("old duplicates:", df_old.duplicated(subset=[key]).sum())
print("new duplicates:", df_new.duplicated(subset=[key]).sum())

161502 159885
Added 6650
Removed 8267
Common 153235
old duplicates: 0
new duplicates: 0


In [3]:
# show 10 example keys that were added / removed / changed
list(added)[:10]
list(removed)[:10]

# sample a few changed keys (if you want to inspect)
def row_by_key(df, k):
    return df[df[key] == k].iloc[0].to_dict()

sample_changed = []
for k in list(common)[:100]:   # check first 100 common to find diffs
    r_old = row_by_key(df_old, k)
    r_new = row_by_key(df_new, k)
    for col in df_old.columns:
        if col == key:
            continue
        if r_old.get(col, "") != r_new.get(col, ""):
            sample_changed.append((k, col, r_old.get(col), r_new.get(col)))
            break
sample_changed[:10]

[('7391182', 'ticket_lowprice', '33.81', '29.03'),
 ('7123016', 'ticket_lowprice', '44.08', '36.63'),
 ('7349978', 'ticket_lowprice', '145.72', '65.10'),
 ('6796220', 'ticket_lowprice', '72.98', '73.76'),
 ('7367674', 'ticket_lowprice', '76.59', '35.65'),
 ('7543392', 'ticket_lowprice', '38.71', '20.68'),
 ('7577592', 'ticket_lowprice', '78.20', '45.92'),
 ('7116580', 'ticket_lowprice', '120.18', '114.70'),
 ('7460478', 'ticket_lowprice', '45.24', '42.08'),
 ('7238235', 'ticket_lowprice', '97.23', '92.86')]

In [4]:
# Create an accurate delta.csv between two CSVs (normalizes values before comparing)
import pandas as pd
from pathlib import Path
import re

# --- Config ---
old_file = "23112025affiliate-98__TicketSqueeze-Events.csv"
new_file = "30112025affiliate-98__TicketSqueeze-Events.csv"
out_file = "delta.csv"
normalize_case = True      # lower-case text before comparing
normalize_whitespace = True
drop_key_duplicates = True # if key duplicates exist, keep first and warn
# ----------------

def detect_key(df):
    for c in df.columns:
        if c.lower() == "event_id":
            return [c]
    for c in df.columns:
        if c.lower() == "id":
            return [c]
    return [df.columns[0]]

def normalize_series(s: pd.Series):
    # operate on strings: strip, collapse spaces, optionally lower
    s = s.fillna("").astype(str)
    if normalize_whitespace:
        s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    else:
        s = s.str.strip()
    if normalize_case:
        s = s.str.lower()
    return s

def normalize_df(df: pd.DataFrame):
    df2 = df.copy()
    for c in df2.columns:
        df2[c] = normalize_series(df2[c])
    return df2

p_old = Path(old_file)
p_new = Path(new_file)
if not p_old.exists() or not p_new.exists():
    raise FileNotFoundError("One of the input files does not exist")

# Read as strings to avoid dtype surprises
df_old = pd.read_csv(p_old, dtype=str).fillna("")
df_new = pd.read_csv(p_new, dtype=str).fillna("")

key_cols = detect_key(df_old)
print(f"Detected key columns: {key_cols}")

# Ensure key exists
for k in key_cols:
    if k not in df_old.columns or k not in df_new.columns:
        raise ValueError(f"Key column '{k}' not found in both files")

# Check duplicates and optionally drop
dups_old = df_old.duplicated(subset=key_cols).sum()
dups_new = df_new.duplicated(subset=key_cols).sum()
if dups_old or dups_new:
    print(f"Warning: duplicates found — old:{dups_old} new:{dups_new}. Using first occurrence.")
    if drop_key_duplicates:
        df_old = df_old.drop_duplicates(subset=key_cols, keep='first')
        df_new = df_new.drop_duplicates(subset=key_cols, keep='first')

# Keep original copies for output
df_old_orig = df_old.copy()
df_new_orig = df_new.copy()

# Normalize copies for robust comparisons
df_old_norm = normalize_df(df_old)
df_new_norm = normalize_df(df_new)

# Build lookup dicts: key-tuple -> row (Series)
def key_of_row(row, keys):
    return tuple(row[k] for k in keys)

old_map_norm = { key_of_row(r, key_cols): r for _, r in df_old_norm.iterrows() }
new_map_norm = { key_of_row(r, key_cols): r for _, r in df_new_norm.iterrows() }
old_map_orig = { key_of_row(r, key_cols): r for _, r in df_old_orig.iterrows() }
new_map_orig = { key_of_row(r, key_cols): r for _, r in df_new_orig.iterrows() }

old_keys = set(old_map_norm.keys())
new_keys = set(new_map_norm.keys())
added_keys = sorted(new_keys - old_keys)
removed_keys = sorted(old_keys - new_keys)
common_keys = sorted(old_keys & new_keys)

print(f"Old rows: {len(df_old)}, New rows: {len(df_new)}")
print(f"Added: {len(added_keys)}, Removed: {len(removed_keys)}, Common: {len(common_keys)}")

records = []
# Added: include original new row prefixed 'new_'
if added_keys:
    rows = [new_map_orig[k] for k in added_keys]
    df_added = pd.DataFrame(rows)
    df_added = df_added.rename(columns={c: f"new_{c}" for c in df_added.columns if c not in key_cols})
    df_added['delta_type'] = 'added'
    records.append(df_added)

# Removed: include original old row prefixed 'old_'
if removed_keys:
    rows = [old_map_orig[k] for k in removed_keys]
    df_removed = pd.DataFrame(rows)
    df_removed = df_removed.rename(columns={c: f"old_{c}" for c in df_removed.columns if c not in key_cols})
    df_removed['delta_type'] = 'removed'
    records.append(df_removed)

# Changed: compare normalized values column-by-column and only mark if any non-key differs
changed_rows = []
for k in common_keys:
    old_row_norm = old_map_norm[k]
    new_row_norm = new_map_norm[k]
    # check any non-key column differs in normalized form
    differs = False
    for col in df_old_norm.columns:
        if col in key_cols:
            continue
        v_old = old_row_norm[col]
        v_new = new_row_norm.get(col, "") if col in df_new_norm.columns else ""
        if pd.isna(v_old) and pd.isna(v_new):
            continue
        if str(v_old) != str(v_new):
            differs = True
            break
    if differs:
        old_row_orig = old_map_orig[k]
        new_row_orig = new_map_orig[k]
        rec = {}
        # key columns kept as-is (original values)
        for kc_idx, kc in enumerate(key_cols):
            rec[kc] = k[kc_idx]
        # add old_ and new_ original columns for visibility
        for col in df_old_orig.columns:
            if col not in key_cols:
                rec[f"old_{col}"] = old_row_orig[col]
        for col in df_new_orig.columns:
            if col not in key_cols:
                rec[f"new_{col}"] = new_row_orig[col]
        rec['delta_type'] = 'changed'
        changed_rows.append(rec)

if changed_rows:
    df_changed = pd.DataFrame(changed_rows)
    records.append(df_changed)

if not records:
    print('No differences found — creating empty delta file')
    pd.DataFrame().to_csv(out_file)
else:
    df_out = pd.concat(records, ignore_index=True, sort=False).fillna("")
    # order columns: keys, delta_type, then the rest
    rest = [c for c in df_out.columns if c not in key_cols + ['delta_type']]
    cols_order = list(key_cols) + ['delta_type'] + rest
    cols_order = [c for c in cols_order if c in df_out.columns]
    df_out = df_out[cols_order]
    df_out.to_csv(out_file, index=False)
    print('\n✓ Wrote delta file:', out_file)
    print('  Added:', len(added_keys))
    print('  Removed:', len(removed_keys))
    print('  Changed:', len(changed_rows))


Detected key columns: ['event_id']
Old rows: 161502, New rows: 159885
Added: 6650, Removed: 8267, Common: 153235
Old rows: 161502, New rows: 159885
Added: 6650, Removed: 8267, Common: 153235

✓ Wrote delta file: delta.csv
  Added: 6650
  Removed: 8267
  Changed: 67071

✓ Wrote delta file: delta.csv
  Added: 6650
  Removed: 8267
  Changed: 67071
